In [31]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Normalization
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd

# Set seeds for reproducibility
tf.random.set_seed(10)
np.random.seed(10)

In [46]:
# Load data
train_df = pd.read_csv('playground-series-s5e5/train.csv')

train_df

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
0,0,male,36,189.0,82.0,26.0,101.0,41.0,150.0
1,1,female,64,163.0,60.0,8.0,85.0,39.7,34.0
2,2,female,51,161.0,64.0,7.0,84.0,39.8,29.0
3,3,male,20,192.0,90.0,25.0,105.0,40.7,140.0
4,4,female,38,166.0,61.0,25.0,102.0,40.6,146.0
...,...,...,...,...,...,...,...,...,...
749995,749995,male,28,193.0,97.0,30.0,114.0,40.9,230.0
749996,749996,female,64,165.0,63.0,18.0,92.0,40.5,96.0
749997,749997,male,60,162.0,67.0,29.0,113.0,40.9,221.0
749998,749998,male,45,182.0,91.0,17.0,102.0,40.3,109.0


In [45]:


# Encode 'Sex' column: male=1, female=0
train_df['Sex'] = train_df['Sex'].map({'male': 1, 'female': 0})

# Features and target
X = train_df.drop(columns=['id', 'Calories']).values
y = train_df['Calories'].values

# Split into train and validation
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.1, random_state=42
)

# Create normalization layer and adapt on training features
normalizer = Normalization()
normalizer.adapt(X_train)

In [28]:
input_layer = Input(shape=(X_train.shape[1],))
x = normalizer(input_layer)

# Branch 1
x1 = Dense(256, activation='relu')(x)
x1 = Dense(128, activation='sigmoid')(x1)
x1 = Dense(32, activation='tanh')(x1)

# Branch 2
x2 = Dense(32, activation='sigmoid')(x)
x2 = Dense(16, activation='tanh')(x2)

combined = concatenate([x1, x2])
output = Dense(1, activation='relu')(combined)

model = Model(inputs=input_layer, outputs=output)

In [29]:
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 7)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_3     │ (None, 7)         │         15 │ input_layer_4[0]… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_23 (Dense)    │ (None, 256)       │      2,048 │ normalization_3[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_24 (Dense)    │ (None, 128)       │     32,896 │ dense_23[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_26 (Dense)    │ (None, 32)        │        256 │ normalization_3[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_25 (Dense)    │ (None, 32)        │      4,128 │ dense_24[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_27 (Dense)    │ (None, 16)        │        528 │ dense_26[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_3       │ (None, 48)        │          0 │ dense_25[0][0],   │
│ (Concatenate)       │                   │            │ dense_27[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_28 (Dense)    │ (None, 1)         │         49 │ concatenate_3[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 39,920 (155.94 KB)

 Trainable params: 39,905 (155.88 KB)

 Non-trainable params: 15 (64.00 B)

In [32]:

def rmsle(y_true, y_pred):
    # Clip values to avoid log of zero or negative numbers
    y_true = tf.clip_by_value(y_true, 0, tf.float32.max)
    y_pred = tf.clip_by_value(y_pred, 0, tf.float32.max)
    
    # Calculate log(1 + y) for both true and predicted
    log_true = tf.math.log1p(y_true)
    log_pred = tf.math.log1p(y_pred)
    
    # Compute squared differences and mean
    return tf.sqrt(tf.reduce_mean(tf.square(log_pred - log_true)))

In [33]:
model.compile(optimizer=Adam(learning_rate=0.0004), loss=rmsle)

callbacks = [
    EarlyStopping(monitor='val_loss', patience=5,restore_best_weights=True,min_delta=1e-4, mode='min',verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6, mode='min', verbose=1),
]

for _ in range(10):
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=200,
        batch_size=256,
        callbacks=callbacks,
        verbose=1
    )

Epoch 1/200
2637/2637 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 1.7520 - val_loss: 0.8602 - learning_rate: 4.0000e-04
Epoch 2/200
1374/2637 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.7979

KeyboardInterrupt: 

In [10]:
input_layer = Input(shape=(X_train.shape[1],))
x = normalizer(input_layer)

# Branch 1
x1 = Dense(128, activation='relu')(x)
x1 = Dense(64, activation='relu')(x1)
x1 = Dense(32, activation='relu')(x1)

# Branch 2
x2 = Dense(128, activation='relu')(x)
x2 = Dense(64, activation='relu')(x2)

combined = concatenate([x1, x2])
output = Dense(1, activation='relu')(combined)

model1 = Model(inputs=input_layer, outputs=output)

In [16]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 7)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_1     │ (None, 7)         │         15 │ input_layer_1[0]… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 256)       │      2,048 │ normalization_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 128)       │     32,896 │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 32)        │        256 │ normalization_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 32)        │      4,128 │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 16)        │        528 │ dense_8[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 48)        │          0 │ dense_7[0][0],    │
│ (Concatenate)       │                   │            │ dense_9[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 1)         │         49 │ concatenate[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 119,732 (467.71 KB)

 Trainable params: 39,905 (155.88 KB)

 Non-trainable params: 15 (64.00 B)

 Optimizer params: 79,812 (311.77 KB)

In [11]:
model1.compile(optimizer=Adam(learning_rate=0.0003), loss=rmsle)

callbacks = [
    EarlyStopping(monitor='val_loss', patience=5,restore_best_weights=True,min_delta=1e-4, mode='min',verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6, mode='min', verbose=1),
]

for _ in range(10):
    history = model1.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=200,
        batch_size=128,
        callbacks=callbacks,
        verbose=1
    )

Epoch 1/200
5274/5274 ━━━━━━━━━━━━━━━━━━━━ 5s 812us/step - loss: 0.5748 - val_loss: 0.0614 - learning_rate: 3.0000e-04
Epoch 2/200
5274/5274 ━━━━━━━━━━━━━━━━━━━━ 4s 762us/step - loss: 0.0608 - val_loss: 0.0599 - learning_rate: 3.0000e-04
Epoch 3/200
5274/5274 ━━━━━━━━━━━━━━━━━━━━ 4s 769us/step - loss: 0.0598 - val_loss: 0.0596 - learning_rate: 3.0000e-04
Epoch 4/200
5274/5274 ━━━━━━━━━━━━━━━━━━━━ 4s 759us/step - loss: 0.0594 - val_loss: 0.0594 - learning_rate: 3.0000e-04
Epoch 5/200
5274/5274 ━━━━━━━━━━━━━━━━━━━━ 4s 757us/step - loss: 0.0592 - val_loss: 0.0594 - learning_rate: 3.0000e-04
Epoch 6/200
5263/5274 ━━━━━━━━━━━━━━━━━━━━ 0s 717us/step - loss: 0.0591
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0001500000071246177.
5274/5274 ━━━━━━━━━━━━━━━━━━━━ 4s 768us/step - loss: 0.0591 - val_loss: 0.0594 - learning_rate: 3.0000e-04
Epoch 7/200
5274/5274 ━━━━━━━━━━━━━━━━━━━━ 4s 760us/step - loss: 0.0582 - val_loss: 0.0596 - learning_rate: 1.5000e-04
Epoch 8/200
5229/5274 ━━━━━━━━━

In [12]:
# Load test data
test_df = pd.read_csv('playground-series-s5e5/test.csv')

# Map 'Sex' in test set: male=1, female=0
test_df['Sex'] = test_df['Sex'].map({'male': 1, 'female': 0})

# Drop 'id' and get feature values
X_test = test_df.drop(columns=['id']).values.astype(np.float32)

# Normalize test features using the pre-fitted normalizer
X_test_normalized = normalizer(X_test)


In [13]:
# Predictions
preds_model = model.predict(X_test_normalized, verbose=0)
preds_model1 = model1.predict(X_test_normalized, verbose=0)

# Average ensemble
final_preds = (preds_model + preds_model1) / 2


In [14]:
final_preds

array([[72.45954],
       [72.49972],
       [68.75578],
       ...,
       [69.3483 ],
       [69.65143],
       [73.66996]], dtype=float32)

In [44]:
# Clip predictions to avoid negative values
final_test_preds_clipped = np.clip(final_preds, 0, None).flatten()

# Create submission DataFrame
submission_df = pd.DataFrame({
    'id': test_df['id'],
    'Calories': final_test_preds_clipped
})

# Save to CSV without index
submission_df.to_csv('submission.csv', index=False)

print("Submission file 'submission.csv' created successfully.")


Submission file 'submission.csv' created successfully.


In [43]:
from sklearn.metrics import mean_squared_log_error
import numpy as np

# Normalize validation data
X_val_normalized = normalizer(X_val)

# Predict from both models
val_preds_model = model.predict(X_val_normalized, verbose=0)
val_preds_model1 = model1.predict(X_val_normalized, verbose=0)

# Ensemble (average predictions)
final_val_preds = (val_preds_model + val_preds_model1) / 2

# Clip to avoid log(0)
final_val_preds_clipped = np.clip(final_val_preds, 0, None)
y_val_clipped = np.clip(y_val, 0, None)

# Compute RMSLE
rmsle_score = np.sqrt(mean_squared_log_error(y_val_clipped, final_val_preds_clipped))
print(f"Validation RMSLE: {rmsle_score:.5f}")


Validation RMSLE: 0.97916


In [42]:
from sklearn.metrics import mean_squared_log_error
import numpy as np

# Normalize training data
X_train_normalized = normalizer(X_train)

# Predict with both models
train_preds_model = model.predict(X_train_normalized, verbose=0)
train_preds_model1 = model1.predict(X_train_normalized, verbose=0)

# Ensemble
final_train_preds = (train_preds_model + train_preds_model1) / 2

# Clip to avoid log(0)
final_train_preds_clipped = np.clip(final_train_preds, 0, None)
y_train_clipped = np.clip(y_train, 0, None)

# Calculate RMSLE
rmsle_train = np.sqrt(mean_squared_log_error(y_train_clipped, final_train_preds_clipped))
print(f"Train RMSLE: {rmsle_train:.5f}")


Train RMSLE: 0.97749
